通用方式之REACT模式比通用方式之FUNCTION模式需要传递更多的必要参数，提示词模板里要包含：
可以使用的工具、用户输入、agent_scratchpad

体会1：使用PromptTemplate

In [4]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent,AgentExecutor,Tool
from langchain_core.prompts import PromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
import os
import dotenv
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["OPENAI_BASE_URL"]=os.getenv("OPENAI_BASE_URL")

template="""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""
prompt_template=PromptTemplate.from_template(template=template)

search=TavilySearchResults(max_results=5)
search_tool=Tool(name="Tavily Search",func=search.run,description="用于查询互联网上的信息")

llm=ChatOpenAI(model="gpt-4o-mini",temperature=0)

agent=create_react_agent(llm=llm,tools=[search_tool],prompt=prompt_template)
agent_executor=AgentExecutor(agent=agent,tools=[search_tool],verbose=True,handle_parsing_errors= True)
result=agent_executor.invoke({"input":"北京今天的天气怎么样？"})
print(result)



> Entering new AgentExecutor chain...
Action: Tavily Search  
Action Input: "北京今天的天气"  [{'title': '【北京天气预报】北京天气预报7天,10天,15天_全国天气网', 'url': 'https://tianqi.so.com/weather/', 'content': '# 全国天气网\n\n 首页\n 国内天气\n 空气质量\n 国际天气\n 景点天气\n 天气新闻\n 专业天气\n\n收藏\n\n北京\n\n[切换]\n\n当前时间：2025-12-22周一06:15\n\n空气 良东北风 2级\n\n 今天 (12-22)\n\n  北风 微风\n\n 明天 (12-23)\n\n  西北风 微风\n\n 周三 (12-24)\n\n  晴\n\n  西南风 微风\n\n 周四 (12-25)\n\n  晴\n\n  西南风 微风\n\n 周五 (12-26)\n\n  北风 微风\n\n 周六 (12-27)\n\n  晴\n\n  -6/3℃\n\n  北风 微风\n\n 周日 (12-28)\n\n  晴转多云\n\n  -5/3℃\n\n  北风 微风\n\n 周一 (12-29)\n\n  多云转晴\n\n  -3/6℃\n\n  南风 微风\n\n 周二 (12-30)\n\n  晴转多云\n\n  -4/5℃\n\n  西北风 微风\n\n 周三 (12-31)\n\n  阴\n\n  -4/6℃\n\n  北风 微风\n\n 周四 (01-01)\n\n  阴转多云\n\n  -4/4℃\n\n  东风 微风\n\n 周五 (01-02)\n\n  多云转晴\n\n  -4/5℃\n\n  东风 微风 [...] 周六 (01-03)\n\n  多云转阴\n\n  -5/7℃\n\n  东北风 微风\n\n 周日 (01-04)\n\n  阴\n\n  -4/2℃\n\n  西南风 微风\n\n45日天气\n\n15日天气\n\n今日天气\n\n明日天气\n\n### 当前天气信息\n\n天气：晴 -5℃\n\n体感：-6℃\n\n风向：东北风\n\n风力：2级\n\n气压：1023\n\n湿度：68%\n\n日出：07:32\n\n日落：16

上面使用提示词模板的方式可以简化

In [7]:

from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent,AgentExecutor,Tool
from langchain_community.tools.tavily_search import TavilySearchResults
import os
import dotenv
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["OPENAI_BASE_URL"]=os.getenv("OPENAI_BASE_URL")

# template="""
# Answer the following questions as best you can. You have access to the following tools:
#
# {tools}
#
# Use the following format:
#
# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question
#
# Begin!
#
# Question: {input}
# Thought:{agent_scratchpad}
# """
prompt_template=hub.pull("hwchase17/react")

search=TavilySearchResults(max_results=5)
search_tool=Tool(name="Tavily Search",func=search.run,description="用于查询互联网上的信息")

llm=ChatOpenAI(model="gpt-4o-mini",temperature=0)

agent=create_react_agent(llm=llm,tools=[search_tool],prompt=prompt_template)
agent_executor=AgentExecutor(agent=agent,tools=[search_tool],verbose=True,handle_parsing_errors= True)
result=agent_executor.invoke({"input":"北京今天的天气怎么样？"})
print(result)



> Entering new AgentExecutor chain...
我需要查找北京今天的天气信息。  
Action: Tavily Search  
Action Input: "北京今天的天气"  [{'title': '【北京天气预报】北京天气预报7天,10天,15天_全国天气网', 'url': 'https://tianqi.so.com/weather/', 'content': '# 全国天气网\n\n 首页\n 国内天气\n 空气质量\n 国际天气\n 景点天气\n 天气新闻\n 专业天气\n\n收藏\n\n北京\n\n[切换]\n\n当前时间：2025-12-22周一06:15\n\n空气 良东北风 2级\n\n 今天 (12-22)\n\n  北风 微风\n\n 明天 (12-23)\n\n  西北风 微风\n\n 周三 (12-24)\n\n  晴\n\n  西南风 微风\n\n 周四 (12-25)\n\n  晴\n\n  西南风 微风\n\n 周五 (12-26)\n\n  北风 微风\n\n 周六 (12-27)\n\n  晴\n\n  -6/3℃\n\n  北风 微风\n\n 周日 (12-28)\n\n  晴转多云\n\n  -5/3℃\n\n  北风 微风\n\n 周一 (12-29)\n\n  多云转晴\n\n  -3/6℃\n\n  南风 微风\n\n 周二 (12-30)\n\n  晴转多云\n\n  -4/5℃\n\n  西北风 微风\n\n 周三 (12-31)\n\n  阴\n\n  -4/6℃\n\n  北风 微风\n\n 周四 (01-01)\n\n  阴转多云\n\n  -4/4℃\n\n  东风 微风\n\n 周五 (01-02)\n\n  多云转晴\n\n  -4/5℃\n\n  东风 微风 [...] 周六 (01-03)\n\n  多云转阴\n\n  -5/7℃\n\n  东北风 微风\n\n 周日 (01-04)\n\n  阴\n\n  -4/2℃\n\n  西南风 微风\n\n45日天气\n\n15日天气\n\n今日天气\n\n明日天气\n\n### 当前天气信息\n\n天气：晴 -5℃\n\n体感：-6℃\n\n风向：东北风\n\n风力：2级\n\n气压：1023\n\n湿度：68%\n\

体会2：使用ChatPromptTemplate

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_react_agent,AgentExecutor,Tool
from langchain_community.tools.tavily_search import TavilySearchResults
import os
import dotenv
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["OPENAI_BASE_URL"]=os.getenv("OPENAI_BASE_URL")

prompt_template=ChatPromptTemplate.from_messages([
    ("system", """
    Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"""),
    ("human", "{question}"),
    ("ai", "Thought:{agent_scratchpad}")
])

search=TavilySearchResults(max_results=5)
search_tool=Tool(name="Tavily Search",func=search.run,description="用于查询互联网上的信息")
llm=ChatOpenAI(model="gpt-4o-mini",temperature=0)
agent=create_react_agent(llm=llm,tools=[search_tool],prompt=prompt_template)
agent_executor=AgentExecutor(agent=agent,tools=[search_tool],verbose=True,handle_parsing_errors= True)
result=agent_executor.invoke({"question":"北京今天的天气怎么样？"})
print(result)



> Entering new AgentExecutor chain...
我需要查询今天北京的天气情况。  
Action: Tavily Search  
Action Input: "北京今天的天气"  [{'title': '【北京天气预报】北京天气预报7天,10天,15天_全国天气网', 'url': 'https://tianqi.so.com/weather/', 'content': '# 全国天气网\n\n 首页\n 国内天气\n 空气质量\n 国际天气\n 景点天气\n 天气新闻\n 专业天气\n\n收藏\n\n北京\n\n[切换]\n\n当前时间：2025-12-22周一06:15\n\n空气 良东北风 2级\n\n 今天 (12-22)\n\n  北风 微风\n\n 明天 (12-23)\n\n  西北风 微风\n\n 周三 (12-24)\n\n  晴\n\n  西南风 微风\n\n 周四 (12-25)\n\n  晴\n\n  西南风 微风\n\n 周五 (12-26)\n\n  北风 微风\n\n 周六 (12-27)\n\n  晴\n\n  -6/3℃\n\n  北风 微风\n\n 周日 (12-28)\n\n  晴转多云\n\n  -5/3℃\n\n  北风 微风\n\n 周一 (12-29)\n\n  多云转晴\n\n  -3/6℃\n\n  南风 微风\n\n 周二 (12-30)\n\n  晴转多云\n\n  -4/5℃\n\n  西北风 微风\n\n 周三 (12-31)\n\n  阴\n\n  -4/6℃\n\n  北风 微风\n\n 周四 (01-01)\n\n  阴转多云\n\n  -4/4℃\n\n  东风 微风\n\n 周五 (01-02)\n\n  多云转晴\n\n  -4/5℃\n\n  东风 微风 [...] 周六 (01-03)\n\n  多云转阴\n\n  -5/7℃\n\n  东北风 微风\n\n 周日 (01-04)\n\n  阴\n\n  -4/2℃\n\n  西南风 微风\n\n45日天气\n\n15日天气\n\n今日天气\n\n明日天气\n\n### 当前天气信息\n\n天气：晴 -5℃\n\n体感：-6℃\n\n风向：东北风\n\n风力：2级\n\n气压：1023\n\n湿度：68%\n\